In [36]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
from tkinter import filedialog, messagebox,scrolledtext
from tkinter.filedialog import askopenfilename
import joblib
import tkinter as tk

In [39]:
def predecir_rendimiento(archivo_excel_df):
  modelo = joblib.load('modelo_entrenado_xgb.pkl')
  columnas_entrada = ['Hours_Studied', 'Attendance', 'Parental_Involvement',
    'Access_to_Resources', 'Extracurricular_Activities', 'Sleep_Hours',
    'Motivation_Level', 'Internet_Access', 'Tutoring_Sessions',
    'Family_Income', 'Teacher_Quality', 'School_Type',
    'Learning_Disabilities', 'Parental_Education_Level', 'Distance_from_Home']

  mapeo_columnas = {
    'Horas de estudio': 'Hours_Studied',
    'Porcentaje Asistencias': 'Attendance',
    'Participación de los padres': 'Parental_Involvement',
    'Acceso a recursos': 'Access_to_Resources',
    'Actividades Extracurriculares': 'Extracurricular_Activities',
    'Horas de sueño': 'Sleep_Hours',
    'Nivel de motivación': 'Motivation_Level',
    'Acceso a internet': 'Internet_Access',
    'Sesiones de tutoria': 'Tutoring_Sessions',
    'Ingresos familiares': 'Family_Income',
    'Calidad docente': 'Teacher_Quality',
    'Tipo escuela origen': 'School_Type',
    'Discapacidad Cognitiva': 'Learning_Disabilities',
    'Educación parental': 'Parental_Education_Level',
    'Distancia de casa': 'Distance_from_Home'
    }
  df_original = archivo_excel_df.copy()
  archivo_excel_df = df_original.rename(columns=mapeo_columnas)

  columnas_eliminar = ['Id Estudiante', 'Nombres', 'Apellidos']
  archivo_excel_df = archivo_excel_df.drop(columns=[col for col in columnas_eliminar if col in archivo_excel_df.columns], errors='ignore')

  ordinal_maps = {
    'Parental_Involvement': {'Bajo': 0, 'Medio': 50, 'Alto': 100},
    'Access_to_Resources': {'Bajo': 0, 'Medio': 50, 'Alto': 100},
    'Motivation_Level': {'Bajo': 0, 'Medio': 50, 'Alto': 100},
    'Extracurricular_Activities': {'No': 0, 'Si': 100},
    'Internet_Access': {'No': 0, 'Si': 100},
    'Family_Income': {'Bajo': 0, 'Medio': 50, 'Alto': 100},
    'Teacher_Quality': {'Bajo': 0, 'Medio': 50, 'Alto': 100},
    'School_Type': {'Publica': 25, 'Privada': 75},
    'Learning_Disabilities': {'No': 100, 'Si': 0}, 
    'Parental_Education_Level': {'Secundaria completa': 25, 'Bachiller': 50, 'Titulado': 100},
    'Distance_from_Home': {'Lejos': 0, 'Moderado': 50, 'Cerca': 100}}

  for col, mapping in ordinal_maps.items():
      archivo_excel_df[col] = archivo_excel_df[col].map(mapping)

  if not all(col in archivo_excel_df.columns for col in columnas_entrada):
      raise ValueError("El archivo no contiene todas las columnas requeridas.")
  
  predicciones = modelo.predict(archivo_excel_df[columnas_entrada])
  
  df_original['Rendimiento_Predicho'] = predicciones

  return df_original



In [43]:
def mostrar_resultado(df_resultado):
    ventana_resultado = tk.Toplevel(root)
    ventana_resultado.title("Resultado de la Predicción")
    ventana_resultado.geometry("600x400")

    st = scrolledtext.ScrolledText(ventana_resultado, width=80, height=20)
    st.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

    st.insert(tk.END, df_resultado[['Id Estudiante', 'Nombres', 'Apellidos', 'Rendimiento_Predicho']].head(15).to_string())
    st.configure(state='disabled')  # Para que no se pueda editar
    
    def guardar_archivo():
        ruta = filedialog.asksaveasfilename(
            initialfile="resultados.xlsx",
            defaultextension=".xlsx",
            filetypes=[("Archivos Excel", "*.xlsx")],
            title="Guardar archivo Excel"
        )
        if ruta:
            try:
                df_resultado.to_excel(ruta, index=False)
                messagebox.showinfo("Guardado", f"Archivo guardado en:\n{ruta}")
            except Exception as e:
                messagebox.showerror("Error", f"No se pudo guardar el archivo:\n{e}")

    btn_guardar = tk.Button(ventana_resultado, text="Guardar resultado Excel", command=guardar_archivo)
    btn_guardar.pack(pady=10)

In [41]:
def cargar_archivo():
    filepath = filedialog.askopenfilename(
        title="Selecciona un archivo Excel",
        filetypes=[("Archivos Excel", "*.xlsx *.xls")]
    )
    if not filepath:
        messagebox.showwarning("Atención", "No se seleccionó ningún archivo.")
        return

    try:
        df = pd.read_excel(filepath)
        messagebox.showinfo("Archivo cargado", f"Archivo cargado correctamente:\n{filepath}")
        resultado = predecir_rendimiento(df)
        mostrar_resultado(resultado)
    except Exception as e:
        messagebox.showerror("Error", f"No se pudo cargar el archivo:\n{e}")

In [46]:
root = tk.Tk()
root.title("Cargar Excel para Predicción")
root.geometry("400x150")

btn_cargar = tk.Button(root, text="Seleccionar archivo Excel", command=cargar_archivo)
btn_cargar.pack(expand=True)

root.mainloop()